In [ ]:
#mounting google drive 
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
import warnings 
warnings.filterwarnings("ignore")
import tensorflow as tf
import keras
import pandas as pd 
import numpy as np 
# import nltk 
# import spacy 
import matplotlib.pyplot as plt 
import seaborn as sn 
%matplotlib inline 
from sklearn.model_selection import train_test_split 
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.metrics import log_loss
# from sentence_transformers import SentenceTransformer 
# import torch 
# import os
import random 
from sklearn.metrics import accuracy_score,confusion_matrix, f1_score
# import urduhack #for pre processing Urdu Text
np.random.seed(42)
tf.random.set_seed(42)
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
random.seed(42)
#other required libraries will be imported at the time of requirement 

In [ ]:
#reading the data 
df_train=pd.read_csv("/content/drive/MyDrive/IIIT_RANCHI/Train_Clean.csv")
df_test=pd.read_csv("/content/drive/MyDrive/IIIT_RANCHI/Test_Clean.csv")
print("############################################\n Training Data\n")
display(df_train.head(10))
print("############################################\n Testing Data\n")
display(df_test.head(10))

############################################
 Training Data



,anger,disgust,fear,sadness,surprise,happiness,neutral,Sentences,word_count,char_count,word_density
0,0,0,0,1,1,0,0,محبت پردے میں نفرت کرنےوالو,5,23,0.208333
1,0,0,0,0,0,1,0,حامد میر خوشی اس کا ذمہ دار حکومت خوش,9,29,0.300000
2,0,0,0,0,0,0,1,اس نئے سال شام سپاہی قاتل لارنس شکل میں شعبے ب...,135,526,0.256167
3,0,0,0,0,0,0,1,کچھ اس طرح سے فلم جا سکتے لیکن میں سب سے یقینا...,117,435,0.268349
4,0,0,0,0,0,0,1,رون ہاورڈ اس ایڈیٹرز صرف اتنا کرنا جس کا تھا ا...,84,304,0.275410
5,0,0,0,0,0,0,0,نتائج دنیا کو حیرت میں ڈال سکتی ھے نتائج دس اق...,19,69,0.271429
6,0,0,0,0,1,0,0,ہم غصے خراب موڈ خوشی غمی سب میں چپلی کباب,10,32,0.303030
7,0,0,0,1,1,0,0,اب ایسی حیرت وارفتگی کو کیا کہیے دعا کو ہاتھ ا...,14,52,0.264151
8,0,0,0,1,1,0,0,آپ اپنی آنکھوں سے نفرت پٹی اتاریں شائد اندازہ ...,13,55,0.232143
9,0,0,0,0,1,0,0,حیرت,1,4,0.200000


############################################
 Testing Data



,anger,disgust,fear,sadness,surprise,happiness,neutral,Sentences
0,0,0,0,1,0,0,0,بچھڑنے والوں کا کیسے نہ غم کیا جائے بوجھ ایسا ...
1,0,0,0,0,0,0,1,اس خوفناک اذیت تشدد گندگی میں بدترین ڈی تعاون ...
2,0,0,0,0,0,0,1,تمام ناقص جاز رپ آف افسوس میری وقت پسندیدہ ڈائ...
3,0,0,0,0,0,0,1,اس فلم ساتھ راجر ایوری کوئنتن تارتانتینو سے ہد...
4,0,0,0,0,0,0,1,جنگلی جانوروں غیر ملکی خطوں غیر معمولی ثقافتوں...
5,0,0,0,0,0,0,1,لڑکی سیریل بلاتکاری روشندان ذریعے اس گھور رہا ...
6,0,0,0,1,1,0,0,سر خدا کا واسطہ راولپنڈی پے توجہ دلوائیں راجہ ...
7,0,0,0,0,0,0,1,عزیز مجھے میں کہاں سے آغاز کروں والد بھی لڑکی ...
8,0,0,0,1,0,0,0,بہت آسان ھائی ذرا دماغ کو نفرت بغض عینک اتار ک...
9,0,0,0,0,0,0,1,میں نے سپائک ان فلم مختلف عنوان منتخب کیا تھا ...


##  Using Sentence Embeddings for Machine Learning Models

In [ ]:
!pip install sentence_transformers
from sentence_transformers import SentenceTransformer

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 KB 6.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 97.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 80.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 27.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 115.3 MB/s eta 0:00:00
  Created wheel for sentence_transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=abc76e4ed1225a8d18104752459cb18ae0c4e18568f5472153d5948edef14bed
  Stored in directory: /root/.cache/pip/wheels/5e/6f/8c/d88aec621f3f542d26fac0342bef5e693335d125f4e54aeffe
Successfully built sentence_transformers


In [ ]:
model_embed=SentenceTransformer("paraphrase-multilingual-MiniLM-L12-v2")
model_embed.max_seq_length=512#Using the EDA we can say most of the words are less than this length
print("Max Length of Sequences is ------->",model_embed.max_seq_length)

Max Length of Sequences is -------> 512


In [ ]:
print("*******************Encoding Train Sentences*******************")
sentence_embeddings_train=model_embed.encode(df_train["Sentences"])
print("*******************Training Sentences Encodedx*******************")
print("*******************Encoding Test Sentences*******************")
sentence_embeddings_test=model_embed.encode(df_test["Sentences"])
print("*******************Test Sentences Encoded*******************")

*******************Encoding Train Sentences*******************
*******************Training Sentences Encodedx*******************
*******************Encoding Test Sentences*******************
*******************Test Sentences Encoded*******************


In [ ]:
x_train_val=sentence_embeddings_train
labels_train = df_train[["anger","disgust","fear","sadness","surprise","happiness","neutral"]]
labels_test=labels = df_test[["anger","disgust","fear","sadness","surprise","happiness","neutral"]]
y_train_val=labels_train
x_test=sentence_embeddings_test
y_test=labels_test
x_train,x_val,y_train,y_val=train_test_split(x_train_val,y_train_val,test_size=0.2,random_state=42)
x_train.shape,x_val.shape,x_test.shape,y_train.shape,y_val.shape,y_test.shape

((6240, 384), (1560, 384), (1950, 384), (6240, 7), (1560, 7), (1950, 7))

In [ ]:
#Employing Ensemble Learning of Various Machine Learning Classifiers on the Embeddings
from sklearn.tree import DecisionTreeClassifier
# from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC 
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import VotingClassifier
from sklearn import model_selection
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
def build_model(model,mlb_estimator,xtrain,ytrain,xtest,ytest):
    clf = mlb_estimator(model)
    clf.fit(xtrain,ytrain)
    clf_predictions = clf.predict(xtest)
    acc = accuracy_score(ytest,clf_predictions)
    f_1_score = f1_score(ytest,clf_predictions,average="weighted")
    result = {"accuracy:":acc,"f1_score":f_1_score}
    return clf,result
estimators=[]
#adding weak 6 gnb
estimators.append(("GNB1",GaussianNB()))
estimators.append(("GNB2",GaussianNB()))
estimators.append(("GNB3",GaussianNB()))
estimators.append(("GNB4",GaussianNB()))
estimators.append(("GNB5",GaussianNB()))
estimators.append(("GNB6",GaussianNB()))
#adding 6 weak svc 
estimators.append(("LSVC1",LinearSVC()))
estimators.append(("LSVC2",LinearSVC()))
estimators.append(("LSVC3",LinearSVC()))
estimators.append(("LSVC4",LinearSVC()))
estimators.append(("LSVC5",LinearSVC()))
estimators.append(("LSVC6",LinearSVC()))
#adding 6 weak dtc
estimators.append(("DTC1",DecisionTreeClassifier()))
estimators.append(("DTC2",DecisionTreeClassifier()))
estimators.append(("DTC3",DecisionTreeClassifier()))
estimators.append(("DTC4",DecisionTreeClassifier()))
estimators.append(("DTC5",DecisionTreeClassifier()))
estimators.append(("DTC6",DecisionTreeClassifier()))
#adding 6 weak knn
estimators.append(("KNN1",KNeighborsClassifier()))
estimators.append(("KNN2",KNeighborsClassifier()))
estimators.append(("KNN3",KNeighborsClassifier()))
estimators.append(("KNN4",KNeighborsClassifier()))
estimators.append(("KNN5",KNeighborsClassifier()))
estimators.append(("KNN6",KNeighborsClassifier()))
#adding 6 weak LDA classfiers 
estimators.append(("LDA1",LinearDiscriminantAnalysis()))
estimators.append(("LDA2",LinearDiscriminantAnalysis()))
estimators.append(("LDA3",LinearDiscriminantAnalysis()))
estimators.append(("LDA4",LinearDiscriminantAnalysis()))
estimators.append(("LDA5",LinearDiscriminantAnalysis()))
estimators.append(("LDA6",LinearDiscriminantAnalysis()))

In [ ]:
#defining the ensemble model
!pip install scikit-multilearn
from skmultilearn.problem_transform import LabelPowerset


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.4/89.4 KB 6.2 MB/s eta 0:00:00


In [ ]:
ensemble_model,result=build_model(VotingClassifier(estimators),LabelPowerset,x_train,y_train,x_test,y_test)

In [ ]:
ensemble_model

LabelPowerset(classifier=VotingClassifier(estimators=[('GNB1', GaussianNB()),
                                                      ('GNB2', GaussianNB()),
                                                      ('GNB3', GaussianNB()),
                                                      ('GNB4', GaussianNB()),
                                                      ('GNB5', GaussianNB()),
                                                      ('GNB6', GaussianNB()),
                                                      ('LSVC1', LinearSVC()),
                                                      ('LSVC2', LinearSVC()),
                                                      ('LSVC3', LinearSVC()),
                                                      ('LSVC4', LinearSVC()),
                                                      ('LSVC5', LinearSVC()),
                                                      ('LSVC6', LinearSVC()),
                                                      ('DTC1',
                                                       DecisionTreeClassifier()...
                                                      ('KNN4',
                                                       KNeighborsClassifier()),
                                                      ('KNN5',
                                                       KNeighborsClassifier()),
                                                      ('KNN6',
                                                       KNeighborsClassifier()),
                                                      ('LDA1',
                                                       LinearDiscriminantAnalysis()),
                                                      ('LDA2',
                                                       LinearDiscriminantAnalysis()),
                                                      ('LDA3',
                                                       LinearDiscriminantAnalysis()),
                                                      ('LDA4',
                                                       LinearDiscriminantAnalysis()),
                                                      ('LDA5',
                                                       LinearDiscriminantAnalysis()),
                                                      ('LDA6',
                                                       LinearDiscriminantAnalysis())]),
              require_dense=[True, True])

In [ ]:
result

{'accuracy:': 0.5882051282051282, 'f1_score': 0.6896368154252894}

In [ ]:
from sklearn.model_selection import KFold, cross_val_score
kf =KFold(n_splits=10, shuffle=True, random_state=42)
result_cv_acc=cross_val_score(ensemble_model,x_train,np.array(y_train),scoring="accuracy",cv=kf)
np.mean(result_cv_acc)

0.5918269230769232

In [ ]:
result_cv_f1=cross_val_score(ensemble_model,x_train,y_train,scoring="f1",cv=kf)
np.mean(result_cv_f1)

KeyboardInterrupt: ignored

In [ ]:
print(f"The mean f1 score is {result_cv_f1.mean} and mean accuracy is {result_cv_acc.mean}")

# Back to LSTM

In [ ]:
import numpy as np
import pandas as pd

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import Sequential
from tensorflow.keras.layers import LSTM, Bidirectional, Flatten, Dense, Embedding

from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import CategoricalCrossentropy

In [ ]:
#Splitting the dataset into 80:20 ratio 
x_train_val=df_train["Sentences"]
y_train_val=df_train[["anger","disgust","fear","sadness","surprise","happiness","neutral"]]
x_test=df_test["Sentences"]
y_test=df_test[["anger","disgust","fear","sadness","surprise","happiness","neutral"]]
x_train,x_val,y_train,y_val=train_test_split(x_train_val,y_train_val,test_size=0.2,random_state=42)

In [ ]:
#shape of training data 
print("Shape of train data is\nx_train------>",x_train.shape)
print("y_train------>",y_train.shape)
print("Shape of test data is\nx_test------>",x_test.shape)
print("y_test------>",y_test.shape)
print("Shape of Validation data is\nx_val------>",x_val.shape)
print("y_val------>",y_val.shape)

Shape of train data is
x_train------> (6240,)
y_train------> (6240, 7)
Shape of test data is
x_test------> (1950,)
y_test------> (1950, 7)
Shape of Validation data is
x_val------> (1560,)
y_val------> (1560, 7)


In [ ]:
max_len = 1500
max_features = 5000

In [ ]:
tokenizer = Tokenizer(num_words=max_features, split=' ')
tokenizer.fit_on_texts(x_train_val.values)

x_train_converted = tokenizer.texts_to_sequences(x_train.values)
x_train_converted = pad_sequences(x_train_converted, maxlen=max_len)

x_test_converted = tokenizer.texts_to_sequences(x_test.values)
x_test_converted = pad_sequences(x_test_converted, maxlen=max_len)
x_val_converted = tokenizer.texts_to_sequences(x_val.values)
x_val_converted = pad_sequences(x_val_converted, maxlen=max_len)

In [ ]:
print('The shape of train data :', x_train_converted.shape)
print('The shape of target of the training :', y_train.shape)
print('The shape of test data :', x_test_converted.shape)
print("The shaoe of target of the test: ", y_test.shape)
print("The shape of validation data: ",x_val_converted.shape)
print("The shape of target of validation : ",y_val.shape)

The shape of train data : (6240, 1500)
The shape of target of the training : (6240, 7)
The shape of test data : (1950, 1500)
The shaoe of target of the test:  (1950, 7)
The shape of validation data:  (1560, 1500)
The shape of target of validation :  (1560, 7)


In [ ]:
import numpy as np
import pandas as pd
from keras.layers import Dense, Conv2D, BatchNormalization, LSTM, Bidirectional
from keras.layers import MaxPooling2D
from keras.layers import Input, Flatten, Dropout
from keras.layers import Activation
from keras.optimizers import Adam
from keras.callbacks import LearningRateScheduler, EarlyStopping, ModelCheckpoint
from keras.models import Model, load_model
from keras.utils import to_categorical
# import optuna
import tensorflow as tf
import keras
from tensorflow.keras.optimizers.schedules import ExponentialDecay
# from sentence_transformers import SentenceTransformer 
# from optuna.samplers import TPESampler
# from optuna.integration import TFKerasPruningCallback
from sklearn.model_selection import train_test_split
from sklearn.metrics import balanced_accuracy_score, accuracy_score
from sklearn.metrics import confusion_matrix, classification_report
import time
from pathlib import Path
import random 
np.random.seed(42)
random.seed(42)
import warnings 
warnings.filterwarnings("ignore")

In [ ]:
# def create_lstm_model(trial):

#     x0 = Embedding(max_features, 256, input_length=x_train_converted.shape)
#     lstm_layers = trial.suggest_int('n_layers', 2, 10)
#     lstm_units = np.zeros(lstm_layers, dtype=np.int)
#     lstm_units[0] = trial.suggest_int("lstm_units_L1", 256, 512)
#     lstm = Bidirectional(keras.layers.LSTM(lstm_units[0], return_sequences=True))(x0)
#     for i in range(lstm_layers-1):
#         lstm_units[i+1] = trial.suggest_int("lstm_units_L{}".format(i+2), lstm_units[i]//2, lstm_units[i])
#         lstm = Bidirectional(keras.layers.LSTM(lstm_units[i+1], return_sequences=True))(lstm)    
#     dropout_rate = trial.suggest_float("lstm_dropout", 0.0, 0.3)
#     lstm = Dropout(dropout_rate)(lstm)
#     dense_units = lstm_units[-1]
#     # try different activations
#     activation = trial.suggest_categorical("activation", ["relu", "selu", "elu", "swish"])
#     lstm = Dense(dense_units, activation=activation)(lstm)
#     lstm = Dense(7)(lstm)

#     model = keras.Model(inputs=x0, outputs=lstm)
#     metrics = ["categorical_crossentropy"]
#     model.compile(optimizer="adam", loss="mae", metrics=metrics)
    
#     return model
# # model = keras.Sequential()
# # model.add(Embedding(1024, 1000, input_length=1000))
# # model.add(keras.layers.LSTM(100, dropout=0.2, recurrent_dropout=0.2,return_sequences=True))

# # model.add(Dense(7, activation='softmax'))
# # model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
# EPOCHS = 30 # number of epocs per trial
# BATCH_SIZE=256
# def objective(trial):
    
#     # Clear clutter from previous session graphs.
#     keras.backend.clear_session()
#     strategy=tf.distribute.MirroredStrategy()
#     with strategy.scope():
#         # Generate our trial model.
#         model = create_lstm_model(trial)

#         # learning rate scheduler
#         scheduler = ExponentialDecay(1e-3, 400*((len(x_train_converted)*0.8)/BATCH_SIZE), 1e-5)
#         lr = LearningRateScheduler(scheduler, verbose=0)
    
#         # Fit the model on the training data.
#         # The TFKerasPruningCallback checks for pruning condition every epoch.
#         model.fit(
#             x_train_converted,
#             y_train,
#             batch_size=BATCH_SIZE,
#             callbacks=[TFKerasPruningCallback(trial, "val_loss")],
#             epochs=EPOCHS,
#             validation_data=(x_val_converted, y_val),
#             verbose=1,
#         )

#         # Evaluate the model accuracy on the validation set.
#         score = model.evaluate(x_test_converted,y_test, verbose=0)
#         return score[1]

In [ ]:
# study = optuna.create_study(direction="minimize", sampler=optuna.samplers.TPESampler(), pruner=optuna.pruners.HyperbandPruner())
# study.optimize(objective, n_trials=100)
# pruned_trials = study.get_trials(deepcopy=False, states=[TrialState.PRUNED])
# complete_trials = study.get_trials(deepcopy=False, states=[TrialState.COMPLETE])

In [ ]:
# ### Create sequence
# from tensorflow.keras.preprocessing.sequence import pad_sequences
# from tensorflow.keras.preprocessing.text import Tokenizer
# vocabulary_size = 512
# tokenizer = Tokenizer(num_words= vocabulary_size)
# tokenizer.fit_on_texts(x_train)
# sequences = tokenizer.texts_to_sequences(x_train)
# data = pad_sequences(sequences, maxlen=100)

In [ ]:
from tensorflow.keras.layers import SpatialDropout1D
model = Sequential()
model.add(Embedding(1500, 100,input_length = x_train_converted.shape[1]))
model.add(SpatialDropout1D(0.3))
model.add(LSTM(128,dropout=0.3,recurrent_dropout=0.3))
# model.add(Bidirectional(LSTM(64,return_sequences=True,dropout=0.3,recurrent_dropout=0.3)))
# model.add(LSTM(32, dropout=0.3, recurrent_dropout=0.3))
# model.add(Dense(256,activation="swish"))
# model.add(Dropout(0.5))
model.add(Dense(7,activation='sigmoid'))
model.compile(loss='binary_crossentropy',metrics='accuracy', optimizer='Adam')
print(model.summary())

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_6 (Embedding)     (None, 1500, 100)         150000    
                                                                 
 spatial_dropout1d_6 (Spatia  (None, 1500, 100)        0         
 lDropout1D)                                                     
                                                                 
 lstm_16 (LSTM)              (None, 128)               117248    
                                                                 
 dense_8 (Dense)             (None, 7)                 903       
                                                                 
Total params: 268,151
Trainable params: 268,151
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
# reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1,patience=5,min_lr=1e-05,verbose=1)
early_stoping = EarlyStopping(monitor="val_loss",min_delta=0,patience=5,verbose=1,mode="auto", baseline=None,restore_best_weights=True)
histroy = model.fit(x_train_converted,y_train, validation_data=(x_val_converted,y_val),epochs=100,callbacks=[early_stoping],verbose=1,batch_size=34)

Epoch 1/100
184/184 [==============================] - 1330s 7s/step - loss: 0.3980 - accuracy: 0.4433 - val_loss: 0.2804 - val_accuracy: 0.5891
Epoch 2/100
184/184 [==============================] - 1269s 7s/step - loss: 0.2767 - accuracy: 0.5708 - val_loss: 0.2589 - val_accuracy: 0.6013
Epoch 3/100
184/184 [==============================] - 1270s 7s/step - loss: 0.2465 - accuracy: 0.6154 - val_loss: 0.2333 - val_accuracy: 0.6551
Epoch 4/100
184/184 [==============================] - 1266s 7s/step - loss: 0.2175 - accuracy: 0.6614 - val_loss: 0.2223 - val_accuracy: 0.6782
Epoch 5/100
184/184 [==============================] - 1274s 7s/step - loss: 0.2022 - accuracy: 0.6854 - val_loss: 0.2239 - val_accuracy: 0.6679
Epoch 6/100
184/184 [==============================] - 1276s 7s/step - loss: 0.1922 - accuracy: 0.6915 - val_loss: 0.2235 - val_accuracy: 0.6641
Epoch 7/100
184/184 [==============================] - 1263s 7s/step - loss: 0.1826 - accuracy: 0.7077 - val_loss: 0.2280 - val_ac

In [ ]:
model.save_weights("Lstm_model_taskA_weights.h5")
print("model weights saved")
model.save("Lstm_model_taskA.h5")
print("Model saved")

model weights saved
Model saved


In [ ]:
scores=model.evaluate(x_test_converted,y_test)

61/61 [==============================] - 26s 415ms/step - loss: 0.2196 - accuracy: 0.6836


In [ ]:
scores

[0.21961192786693573, 0.6835897564888]

In [ ]:
ypred = model.predict(x_test_converted, verbose=0)

In [ ]:
ypred

array([[1.9642036e-01, 1.5850015e-01, 1.1821372e-02, ..., 7.2610277e-01,
        3.3398651e-02, 2.7728226e-02],
       [5.8310229e-04, 3.8065363e-04, 3.4911695e-04, ..., 1.3481998e-03,
        1.8469284e-03, 9.9915540e-01],
       [5.0455955e-04, 3.0833748e-04, 3.4937763e-04, ..., 1.4436383e-03,
        1.8858290e-03, 9.9905580e-01],
       ...,
       [1.0669440e-02, 9.4741387e-03, 8.8424809e-02, ..., 1.2911791e-01,
        1.4136997e-01, 4.3644658e-03],
       [2.0321950e-02, 1.8866654e-02, 3.4807943e-02, ..., 5.6272727e-01,
        1.5606505e-01, 7.2818352e-03],
       [5.4625695e-04, 3.4791237e-04, 3.5657029e-04, ..., 1.3973183e-03,
        1.8271923e-03, 9.9907529e-01]], dtype=float32)

In [ ]:
labels=df_test[["anger","disgust","fear","sadness","surprise","happiness","neutral"]]
Test_prob = pd.DataFrame(ypred)
Test_prob.columns = labels.columns

In [ ]:
Test_prob

,anger,disgust,fear,sadness,surprise,happiness,neutral
0,0.196420,0.158500,0.011821,0.363806,0.726103,0.033399,0.027728
1,0.000583,0.000381,0.000349,0.001110,0.001348,0.001847,0.999155
2,0.000505,0.000308,0.000349,0.001163,0.001444,0.001886,0.999056
3,0.000461,0.000276,0.000370,0.001344,0.001446,0.001759,0.998944
4,0.000430,0.000263,0.000390,0.001542,0.001345,0.001468,0.998870
...,...,...,...,...,...,...,...
1945,0.373504,0.289169,0.017001,0.285146,0.588391,0.042715,0.038004
1946,0.000485,0.000313,0.000334,0.001260,0.001483,0.001609,0.999036
1947,0.010669,0.009474,0.088425,0.819834,0.129118,0.141370,0.004364
1948,0.020322,0.018867,0.034808,0.453520,0.562727,0.156065,0.007282


In [ ]:
final_label=Test_prob.round()#rounding of the probabilities to labels
final_label_df=pd.DataFrame(final_label)
final_label_df.columns=labels.columns
final_label_df

,anger,disgust,fear,sadness,surprise,happiness,neutral
0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,0.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...
1945,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1946,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1947,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1948,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [ ]:
final_test=labels
final_test_y1 = final_test.reset_index(drop=True)
print("The F-1 Score is : {:.2f} ".format(f1_score(final_test_y1, final_label_df,average="weighted")))

The F-1 Score is : 0.67 


In [ ]:
print("The F-1 Score is : {:.2f} ".format(f1_score(final_test_y1, final_label_df,average="macro")))

The F-1 Score is : 0.55 


In [ ]:
NUM_FILTERS = 1024
NUM_WORDS = 7
embed_dim = 256
batch_size = 32
EPOCHS = 1
from tensorflow.keras.layers import SpatialDropout1D, Embedding, Conv1D, GlobalMaxPooling1D
model_cnn = keras.Sequential()
model_cnn.add(Embedding(1500, embed_dim, input_length = x_train_converted.shape[1]))
model_cnn.add(SpatialDropout1D(0.2))
model_cnn.add(Conv1D(filters=NUM_FILTERS, kernel_size=NUM_WORDS, activation="relu"))
model_cnn.add(GlobalMaxPooling1D())
model_cnn.add(Dense(7, activation="softmax"))
model_cnn.compile(optimizer="adam", loss="binary_crossentropy",metrics=["accuracy"])
print(model.summary())

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_7 (Embedding)     (None, 1500, 256)         384000    
                                                                 
 spatial_dropout1d_7 (Spatia  (None, 1500, 256)        0         
 lDropout1D)                                                     
                                                                 
 conv1d (Conv1D)             (None, 1494, 1024)        1836032   
                                                                 
 global_max_pooling1d (Globa  (None, 1024)             0         
 lMaxPooling1D)                                                  
                                                                 
 dense_9 (Dense)             (None, 7)                 7175      
                                                                 
Total params: 2,227,207
Trainable params: 2,227,207
No

In [ ]:
model_cnn.fit(x_train_converted, y_train, batch_size=batch_size,epochs=20,validation_data=(x_val_converted,y_val),callbacks=[early_stoping])

Epoch 1/20
195/195 [==============================] - 44s 221ms/step - loss: 0.2813 - accuracy: 0.5902 - val_loss: 0.2193 - val_accuracy: 0.6712
Epoch 2/20
195/195 [==============================] - 37s 191ms/step - loss: 0.1998 - accuracy: 0.6837 - val_loss: 0.2147 - val_accuracy: 0.6865
Epoch 3/20
195/195 [==============================] - 33s 172ms/step - loss: 0.1609 - accuracy: 0.7231 - val_loss: 0.2368 - val_accuracy: 0.6891
Epoch 4/20
195/195 [==============================] - 30s 154ms/step - loss: 0.1213 - accuracy: 0.7622 - val_loss: 0.2483 - val_accuracy: 0.6878
Epoch 5/20
195/195 [==============================] - 28s 144ms/step - loss: 0.0840 - accuracy: 0.7825 - val_loss: 0.2722 - val_accuracy: 0.6776
Epoch 6/20
195/195 [==============================] - 28s 145ms/step - loss: 0.0566 - accuracy: 0.8006 - val_loss: 0.3014 - val_accuracy: 0.6667
Epoch 7/20
195/195 [==============================] - 28s 142ms/step - loss: 0.0409 - accuracy: 0.8093 - val_loss: 0.3413 - val_ac

In [ ]:
scores_cnn=model.evaluate(x_test_converted,y_test)

61/61 [==============================] - 2s 33ms/step - loss: 0.4794 - accuracy: 0.6518


In [ ]:
model.save_weights("cnn_model_taskA_weights.h5")
print("model weights saved")
model.save("cnn_model_taskA.h5")
print("Model saved")

model weights saved
Model saved


In [ ]:
ypred = model.predict(x_test_converted, verbose=0)

In [ ]:
labels=df_test[["anger","disgust","fear","sadness","surprise","happiness","neutral"]]
Test_prob = pd.DataFrame(ypred)
Test_prob.columns = labels.columns

In [ ]:
final_label=Test_prob.round()#rounding of the probabilities to labels
final_label_df=pd.DataFrame(final_label)
final_label_df.columns=labels.columns
final_label_df

,anger,disgust,fear,sadness,surprise,happiness,neutral
0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,0.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...
1945,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1946,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1947,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1948,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [ ]:
final_test=labels
final_test_y1 = final_test.reset_index(drop=True)
print("The F-1 Score is : {:.2f} ".format(f1_score(final_test_y1, final_label_df,average="weighted")))

The F-1 Score is : 0.63 


In [ ]:
print("The F-1 Score is : {:.2f} ".format(f1_score(final_test_y1, final_label_df,average="macro")))

The F-1 Score is : 0.48 
